In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from unidecode import unidecode
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/foursquare-location-matching/train.csv")

In [ ]:
default_nan_string = '__nan__'

cats = train.categories.fillna(default_nan_string)\
    .apply(unidecode)\
    .str.lower()

In [ ]:
cat_stats = {}

for c in cats:
    csplit = c.split(", ")
    for c_ in csplit:
        if c_ not in cat_stats:
            cat_stats[c_] = [1, {ca: 1 for ca in csplit}]
        else:
            prev_cat_stats = cat_stats[c_]
            prev_cat_stats[0] += 1
            prev_sims = prev_cat_stats[1]
            
            for ca in csplit:
                prev_sims[ca] = prev_sims.get(ca, 0) + 1
        
for c, val in cat_stats.items():
    
    overall_count = sum(v for ca, v in val[1].items() if ca != c and v > 1)
    
    new_probs = {c: 1.0}
    if overall_count != 0:
        new_probs.update({ca: round(v / overall_count, 3) for ca, v in val[1].items() if v / overall_count > 0.01 and ca != c and v > 1})
        cat_stats[c] = [1.0, new_probs]
        
def get_cat_similarity(cat1, cat2):
    
    if cat1 == default_nan_string or cat2 == default_nan_string:
        return 0
    
    if cat1 in cat2 or cat2 in cat1:
        return 1.0
    
    best = 0
    
    for c1 in cat1.split(", "):
        for c2 in cat2.split(", "):
            if c1 in cat_stats:
                best = max(best, cat_stats[c1][1].get(c2, 0))
            if c2 in cat_stats:
                best = max(best, cat_stats[c2][1].get(c1, 0))
    
    return best

In [ ]:
cat_stats['bars']

In [ ]:
get_cat_similarity('bars', 'cafes, restaurants')